## Real Estate Recommender

### Generating Real Estate Listings

In [38]:
groups = ["young professionals", "families", "retirees and seniors", "investors", "students", "luxury buyers", "first-time homebuyers", "eco-conscious buyers", "high-net-worth individuals", "low-income families", "digital nomads and remote workers", "artists and creatives"]
user_prompt_template = lambda buyer_group: f"""Task: Create a detailed and appealing real estate property listing for {buyer_group}."""
system_prompt_template = """
                        You are an assistant that generate text without repeatation. Text should follow following format strictly:
                        - Neighborhood: address, city
                        - Price: price
                        - Bedrooms: number of bedrooms
                        - Bathrooms number of bathrooms
                        - House Size: size in sqft
                        - Description: general description of the property
                        - Neighborhood Description: description of neighborhood environments
                    """

In [39]:
# Importing the necessary library for OpenAI API
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [53]:

def generate_resl_estate_listing(buyer_group):
    try:
        client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),
            # I have chosen to use local model host, as OpenAI has recently closed the 5$ api key bonus
            # the model supports same api format as openai does
            base_url="http://localhost:8080"
        )
        response = client.chat.completions.create(
            model="gpt4-turbo",
            # model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": system_prompt_template,
                },
                {
                    "role": "user",
                    "content": user_prompt_template(buyer_group),
                },
            ],
            temperature=1,
            # max_tokens=256,
            top_p=1,
            frequency_penalty=1,
            presence_penalty=1
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"


In [57]:
import pandas as pd
import time

df_list = []
size_per_group = 10
for id, people in enumerate(groups):
    df_list.append(pd.DataFrame({"listing": [generate_resl_estate_listing(user_prompt_template(people)) for i in range(size_per_group)], "group": [id]*size_per_group}))
    time.sleep(10)
df = pd.concat(df_list).reset_index(drop=True)
pd.set_option("display.max_colwidth", None)
df

,listing,group
0,"- Neighborhood: 250 Waterfront St, Marina District\n- Price: $895,000\n- Bedrooms: 2\n- Bathrooms: 2\n- House Size: 1,100 sqft\n- Description: Immerse yourself in the heart of vibrancy and comfort with this stylish modern condo designed for young professionals. The open-plan living space is bathed in natural light from floor-to-ceiling windows that offer enchanting cityscape views. High-end finishes are evident throughout the kitchen that boasts stainless steel appliances, sleek cabinetry, and a chic breakfast bar perfect for on-the-go mornings or intimate dinners. Both bedrooms serve as private havens with ample closet space and plush carpeting underfoot.\n \n Indulge in the luxury of en-suite bathrooms featuring contemporary fixtures and fine tile work that add to the elegance of urban living. As an added convenience, an in-unit washer and dryer set simplifies laundry days.\n\n This home is more than just its interior; it includes access to top-tier building amenities such as a rooftop terrace with panoramic views, state-of-the-art fitness center, residents' lounge for entertaining guests, secured parking structure, and attentive concierge services enhancing daily life.\n\n- Neighborhood Description: Located in the sought-after Marina Districts area often frequented by thriving professionals. The neighborhood pulses with cosmopolitan energy yet retains a relaxed atmosphere conducive to networking and unwinding alike. Within walking distance are gourmet restaurants offering international cuisines, boutique shops for fashion-forward finds along scenic promenades lined with public art installations reflective of local creativity.\n\n Outdoor enthusiasts will appreciate proximity to parks where jogging trails meander alongside waterfront vistas—a tranquil escape from busy schedules. Cultural institutions like theatres hosting premiere performances contribute to rich experiences just steps away from your front door while reliable public transit connects you effortlessly across the city's diverse neighborhoods.\n",0
1,"- Neighborhood: 352 Parkview Terrace, Austin\n- Price: $425,000\n- Bedrooms: 2\n- Bathrooms: 2\n- House Size: 1,200 sqft\n- Description: This sleek and modern condo offers the perfect blend of style and convenience for the young professional. Featuring state-of-the-art appliances, hardwood flooring throughout, an open-concept living space with large windows that flood the area with natural light. The master suite includes a walk-in closet and a pristine en-suite bathroom. Additional amenities include in-unit laundry facilities and private balcony with breathtaking city views.\n \n- Neighborhood Description: Situated in the heart of Austin's vibrant scene, Parkview Terrace is an ideal spot for those looking to be at arm’s reach from bustling cafes, hip restaurants & bars, boutique shopping options as well as premier entertainment venues. The neighborhood boasts easy access to public transportation facilitating commutes. It is also known for its safety measures and pet-friendly parks which just adds another layer of allure to this dynamic community perfect for social but career-focused individuals.",0
2,"- Neighborhood: 255 Upstart Lane, Silicon Valley\n- Price: $1,200,000\n- Bedrooms: 3\n- Bathrooms: 2\n- House Size: 1800 sqft\n- Description: This modern and sleek single-family home offers an open floor plan with high-end finishes throughout. The kitchen boasts stainless steel appliances and a large island perfect for entertaining. Hardwood floors lead you through the light-filled living space to a cozy backyard oasis. Equipped with smart home features and energy-efficient windows.\n- Neighborhood Description: Situated in the heart of Silicon Valley, this neighborhood is vibrant and full of life, catering to young professionals keen on innovation and convenience. Just steps away from tech headquarters, dynamic start-ups, upscale dining options, boutique coffee shops, and lush parks ideal for weekend

### Storing Listings in a Vector Database

In [8]:
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry
import lancedb
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# func = get_registry().get("openai").create(name="text-embedding-ada-002")
import openai
import os

EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    # I have chosen to use local model host, as OpenAI has recently closed the 5$ api key bonus
    # the model supports same api format as openai does
    base_url="http://localhost:8080"
)
def embed_func(c):    
    response = client.embeddings.create(input=c, model=EMBEDDING_MODEL_NAME)
    return [data.embedding for data in response.data]

In [9]:
class RealEstateListing(LanceModel):
    vector: Vector(384)
    listing: str
    group: str

In [17]:
from lancedb.embeddings import with_embeddings
import pandas as pd

df_embedded = with_embeddings(embed_func, df, column="listing").to_pandas()
df_embedded


/tmp/ipykernel_2259183/810152267.py:4: DeprecationWarning: Function with_embeddings is deprecated and will be removed in a future version
  df_embedded = with_embeddings(embed_func, df, column="listing").to_pandas()


,listing,group,vector
0,"- Neighborhood: 250 Waterfront St, Marina Dist...",0,"[0.058508158, 0.029074132, 0.043684043, -0.009..."


In [64]:

db = lancedb.connect("./lancedb")
table_name = "real_estate_listings"
table = db.create_table(table_name, schema=RealEstateListing, mode="overwrite")

[2024-06-22T10:25:13Z WARN  lance::dataset] No existing dataset at /lhome/yuwewan/workspace/Minverva/genAI_udacity/Real_Estate_Agent/lancedb/real_estate_listings.lance, it will be created


In [65]:
table.add(df_embedded.to_dict('records'))
table.head().to_pandas()

,vector,listing,group
0,"[0.058508143, 0.02907416, 0.043684054, -0.009807144, -0.050845005, 0.030122882, -0.060871765, 0.0040336815, -0.025515487, -0.0282995, -0.01119739, -0.111601084, 0.03444756, 0.0011670552, 0.08167276, 0.010128735, 0.11821405, 0.026792387, 0.029855762, 0.051701207, -0.04625208, 0.0046300814, 0.00084731093, -0.035731196, 0.0156841, 0.08610467, 0.01831103, 0.03300818, -0.049790993, 0.011102918, 0.059595834, 0.047849704, 0.028044403, -0.06404407, 0.056364294, -0.042862635, -0.034160275, -0.05080546, 0.007422114, -0.00040552652, -0.042594127, -0.017449502, -0.028738149, 0.050229516, -0.045410205, -0.063050054, -0.003511994, 0.03083161, 0.05919047, -0.07009388, 0.03898609, 0.08206192, 0.030433357, 0.03248369, -0.04558283, 0.024247643, -0.040412884, -0.029736688, 0.034573033, -0.044887252, 0.09426998, 0.016264196, 0.020763775, 0.01299787, 0.023354582, -0.03948777, -0.016023932, 0.0987113, 0.012233934, -0.07733977, -0.032462303, -0.0021667096, 0.047857303, 0.04200995, 0.0057053664, 0.009708946, 0.040949054, -0.019909529, -0.00068721455, 0.0044354647, 0.024635343, -0.017793769, -0.03741448, 0.0146732135, 0.011596321, -0.0041640406, 0.00031836514, -0.019088313, 0.036408745, 0.010967304, 0.025408411, -9.308738e-05, -0.15183586, -0.099948965, 0.024020763, -0.0034210996, -0.1385258, -0.02444478, 0.010562316, 0.012679376, ...]","- Neighborhood: 250 Waterfront St, Marina District\n- Price: $895,000\n- Bedrooms: 2\n- Bathrooms: 2\n- House Size: 1,100 sqft\n- Description: Immerse yourself in the heart of vibrancy and comfort with this stylish modern condo designed for young professionals. The open-plan living space is bathed in natural light from floor-to-ceiling windows that offer enchanting cityscape views. High-end finishes are evident throughout the kitchen that boasts stainless steel appliances, sleek cabinetry, and a chic breakfast bar perfect for on-the-go mornings or intimate dinners. Both bedrooms serve as private havens with ample closet space and plush carpeting underfoot.\n \n Indulge in the luxury of en-suite bathrooms featuring contemporary fixtures and fine tile work that add to the elegance of urban living. As an added convenience, an in-unit washer and dryer set simplifies laundry days.\n\n This home is more than just its interior; it includes access to top-tier building amenities such as a rooftop terrace with panoramic views, state-of-the-art fitness center, residents' lounge for entertaining guests, secured parking structure, and attentive concierge services enhancing daily life.\n\n- Neighborhood Description: Located in the sought-after Marina Districts area often frequented by thriving professionals. The neighborhood pulses with cosmopolitan energy yet retains a relaxed atmosphere conducive to networking and unwinding alike. Within walking distance are gourmet restaurants offering international cuisines, boutique shops for fashion-forward finds along scenic promenades lined with public art installations reflective of local creativity.\n\n Outdoor enthusiasts will appreciate proximity to parks where jogging trails meander alongside waterfront vistas—a tranquil escape from busy schedules. Cultural institutions like theatres hosting premiere performances contribute to rich experiences just steps away from your front door while reliable public transit connects you effortlessly across the city's diverse neighborhoods.\n",0
1,"[0.08807147, -0.031950254, 0.11615688, 0.006585816, -0.09464219, 0.030149572, 0.0077909925, -0.021867914, -0.007555809, -0.032976802, 0.012449364, 0.009467583, 0.0921661, 0.007781201, 0.07434154, 0.00878247, 0.09606928, 0.027645389, 0.024280358, 0.052039545, -0.028513487, 0.00895382, -0.0075076977, -0.017512724, -0.058618948, 0.12708355, 0.01685586, -0.012254036, -0.03175031, 0.0036859997, 0.08263985, 4.6791287e-05, 0.04767689, -0.042601336, 0.043040067, 0.011154155, -0.017736562, -0.068274006, -0.00048933615, -0.028498916, -0.064493924, 0.017040381, 0.000983461, 0.0045599947, -0.06

### Building the User Preference Interface

In [2]:
import lancedb

db = lancedb.connect("./lancedb")
table_name = "real_estate_listings"
table = db.open_table(table_name)
df = table.to_pandas()

/lhome/yuwewan/anaconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
questions = [   
                "How big do you want your house to be?" 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
answers = [
            "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
            "A quiet neighborhood, good local schools, and convenient shopping options.",
            "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
            "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
            "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
        ]

In [4]:

# build an User Portrait embedding could improve the performance
user_prompt_preference = lambda text: f"""Task: extract user preference based on question and answer : {text}."""
system_prompt_preference = """
                        You generate texts about a potential buyer's preference focusing on factors: address, price, number of bedrooms, number of bathrooms, size in sqft, general property information and neighborhood environment.
                    """
def preference_parser(text):
    try:
        client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),
            # I have chosen to use local model host, as OpenAI has recently closed the 5$ api key bonus
            # the model supports same api format as openai does
            base_url="http://localhost:8080"
        )
        response = client.chat.completions.create(
            model="gpt4-turbo",
            # model='gpt-4'
            messages=[
                {
                    "role": "system",
                    "content": system_prompt_preference,
                },
                {
                    "role": "user",
                    "content": user_prompt_preference(text),
                },
            ],
            temperature=1,
            max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

In [5]:
QA=""
for i in range(len(questions)):
    QA = questions[i]+" "+answers[i]+" "
preference_text = preference_parser(QA)
preference_text

"An error occurred: name 'OpenAI' is not defined"

### Searching Based on Preferences

In [6]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd

EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    # I have chosen to use local model host, as OpenAI has recently closed the 5$ api key bonus
    # the model supports same api format as openai does
    base_url="http://localhost:8080"
)
def embed_func(c):    
    response = client.embeddings.create(input=c, model=EMBEDDING_MODEL_NAME)
    return [data.embedding for data in response.data]

In [7]:
preference_embedded = embed_func(preference_text)[0]
df_matched = table.search(preference_embedded).metric("cosine").limit(3).to_pandas()
pd.set_option("display.max_colwidth", None)
df_matched

,vector,listing,group,_distance
0,"[0.08713895, 0.038796484, 0.047830056, 0.052215934, -0.014083989, 0.03542631, -0.04411613, 0.0022076163, -0.010448868, -0.04087193, 0.001308291, -0.050715018, -0.012174658, -0.0057731927, 0.01754327, 0.01845808, 0.1252117, -0.008980836, 0.05134285, 0.02676107, -0.06362143, -0.001512009, 0.068536684, -0.017498085, 0.031181201, 0.11969546, 0.023040814, 0.022712003, 0.004667142, -0.02954358, 0.09870199, 0.05493239, 0.07447989, -0.10149825, 0.08293515, -0.006031372, 0.036697708, -0.058060613, -0.041509427, -0.011703176, 0.020243712, -0.017395707, -0.029418288, -0.016776137, -0.02711659, -0.01846062, -0.01603048, 0.04136155, 0.11701129, -0.07613034, -0.0029757642, 0.06879449, -0.03261393, -0.042755965, -0.012785296, 0.086726956, -0.054810323, -0.0049458756, 0.022028625, 0.019107394, 0.08207081, -0.006592942, -0.029744634, 0.03707265, 0.0326394, -0.010486727, -0.09435591, -0.013500474, 0.02918679, -0.100924365, -0.044241626, 0.003398833, 0.044511892, -0.06763735, 0.023078617, 0.032940812, 0.046177346, -0.0255212, -0.01016633, -0.012427113, 0.00726572, -0.0123684965, -0.02826496, 0.021709913, -0.0023056536, -0.0016049393, 0.0016169642, 0.003967971, 0.019878417, -0.021827564, -0.03629763, 0.02170995, -0.087663144, -0.07780313, 0.001745215, 0.002872307, -0.08114177, -0.0732963, -0.06207685, 0.008155421, ...]","- Neighborhood: 152 Pleasant View Ave, Sunrise City\n- Price: $99,000\n- Bedrooms: 3\n- Bathrooms: 1\n- House Size: 950 sqft\n- Description: This charming, cozy single-family home offers a comfortable living space at an affordable price. Recently renovated with new laminate flooring and fresh paint throughout, it features a compact kitchen equipped with modern appliances, energy-efficient windows for reduced utility bills, and a quaint living room perfect for family gatherings.\n- Neighborhood Description: The community is known for its friendly residents and communal spirit. Sunrise City boasts several well-maintained parks within walking distance, quality schools that cater to low-income families on a budget without compromising education standards, and convenient public transportation options connecting you to local amenities such as markets and healthcare centers.",9,0.842045
1,"[0.08608704, 0.03202741, 0.10670512, 0.10137339, -0.013084327, -0.054547552, -0.013243855, -0.0069567272, -0.031756483, -0.05050679, 0.07241346, -0.029815909, 0.007838759, 0.031782933, 0.023930902, 0.042833038, 0.16149941, -0.023912005, 0.00084577536, -0.0075112996, -0.07945585, -0.017852712, 0.039899, -0.022898262, 0.029718846, 0.06846278, 0.020092448, 0.041926835, -0.0067645567, 0.007843174, 0.06793542, 0.053957164, 0.048671294, -0.051937267, 0.07825692, 0.03368713, 0.038351443, 0.024120718, -0.026145758, 0.0072084954, -0.026212208, -0.044998564, -0.033473793, -0.012233929, -0.032283228, -0.057822865, 0.023324125, -0.007664554, 0.06033601, -0.08405758, 0.05033287, 0.07562917, -0.012169503, 0.007897766, -0.0548601, 0.06575859, -0.062758245, -0.0064699654, 0.013022333, -0.018186178, 0.006578425, 0.023323834, -0.09785793, 0.018433742, 0.004095852, 0.0042047235, -0.042557567, 0.03989814, -0.02876519, -0.06678433, -0.01588088, -0.0061888653, 0.013901859, 0.019446526, 0.100002035, 0.042393662, 0.028936327, 0.041180212, 0.042208757, 0.04292214, 0.04371958, -0.060528375, -0.08983757, 0.024136234, -0.058533948, 0.0046317005, -0.044300687, 0.033241365, 0.03325294, -0.031610884, -0.08057369, 0.0067561707, -0.05536003, -0.06381253, 0.023792114, 0.002687592, -0.077892944, -0.10793557, -0.009334658, 0.04273953, ...]","- Neighborhood: 1525 Affordable Lane, Opportunity City\n- Price: $120,000\n- Bedrooms: 3\n- Bathrooms: 1.5\n- House Size: 1,100 sqft\n- Description: An attractive and budget-friendly single-family home awaits you in the heart of Opportunity City. This cozy residence offers three well-proportioned bedrooms, a convenient half bath for guests and a full bath featuring modern fittings. The kitchen is equipped with ener

### Personalizing Listing Descriptions

In [8]:

%env CMAKE_ARGS="-DLLAMA_METAL=on"
%env FORCE_CMAKE=1

env: CMAKE_ARGS="-DLLAMA_METAL=on"
env: FORCE_CMAKE=1


In [9]:
from langchain_community.llms import LlamaCpp

n_gpu_layers = 1
n_batch = 512 
model_llama = LlamaCpp(
    model_path="/lhome/yuwewan/workspace/NV_Contest/toxicqa-llama2-13b.Q4_0.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,
    verbose=True,
)

llama_model_loader: loaded meta data with 22 key-value pairs and 363 tensors from /lhome/yuwewan/workspace/NV_Contest/toxicqa-llama2-13b.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 

In [10]:
from langchain_openai import ChatOpenAI, AzureChatOpenAI

base_url = "http://localhost:8080/v1"
model = ChatOpenAI(
    model="gpt4-turbo", 
    openai_api_base=base_url,
    temperature=0,
)

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

index = 0
def get_context(info):
    global index
    if "negative" == info:
        if index < (df_matched.shape[0]-1):
            index += 1
        else:
            return "Sorry, I could not find a match for your!"
    return df_matched.loc[index, "listing"]

feedback_chain = (
    PromptTemplate.from_template(
        """Given the user message below, you estimate whether user is satisfied with the current recommendation, please classify it only with "positive", "neutral" or "negative". Do not respond with more than one word.
        <message>
        {message}
        </message>
        Classification:"""
        )
        | model
        | StrOutputParser()
    )


user_prompt_personalization = """
                        Answer the question based on the context below in a compelling, selling and reader-friendly way without altering factual information.
                        If {feedback} is negative, please resummarize the context text".
                        
                        Context: {context}.
                        ----
                        Question: {message}
                        Answer:
                    """

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a real estate selling assistance, generate text to match the buyer's preference at best.",
        ),
        ("human", user_prompt_personalization),
    ]
)

rag_chain = prompt | model | StrOutputParser()

full_chain = {"feedback":feedback_chain, "context": feedback_chain | get_context, "message": lambda x: x ["message"]} | rag_chain


In [12]:
question = "do you have any recommandation?"
for r in full_chain.stream({"message": question}):
    print(r, end="")

Absolutely, I would highly recommend considering the delightful single-family home located at 152 Pleasant View Ave in the welcoming community of Sunrise City. With its very competitive price tag of just $99,000, this gem offers both affordability and comfort.

This home strikes the perfect balance with three cozy bedrooms and a single bathroom, all within a practical 950 sqft layout. It has been thoughtfully updated to provide a fresh and modern feel, with new laminate flooring and a sleek coat of paint throughout the house. The compact kitchen comes fully equipped with the latest appliances, ideal for culinary enthusiasts or busy family life.

Energy efficiency is a key feature here, with upgraded windows ensuring your utility bills stay low while keeping your living space comfortably regulated throughout the seasons. The living room, with its warm and inviting ambiance, is the perfect backdrop for memorable family moments and relaxation.

The surrounding neighborhood of Sunrise City

In [127]:
question_1 = "how much does it cost?"
for r in full_chain.stream({"message": question_1}):
    print(r, end="")

'The inviting home you are inquiring about at 152 Maple Avenue, Springfield, is priced at an affordable $89,000. This presents an exceptional value for a well-maintained, move-in ready property in a family-friendly neighborhood.'

In [128]:
question_2 = "is a garage inclusive?"
for r in full_chain.stream({"message": question_2}):
    print(r, end="")

'Sorry, I do not have information about that!'